In [4]:
import os
import torch
import torchvision
from groundingdino.util.inference import load_model, load_image, annotate, predict as _predict
import cv2
import tqdm

In [5]:
model = load_model("GroundingDINO_SwinT_OGC.py", "groundingdino_swint_ogc.pth")

C:\Program Files\Python39\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


C:\Program Files\Python39\lib\site-packages\transformers\modeling_utils.py:433: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining m

In [6]:
def predict(image, caption, box_threshold, text_threshold, save=False):
    fname = image.split(os.sep)[-1]
    image_source, image = load_image(image)
    boxes, confidence, phrases = _predict(
    model=model,
    image=image,
    caption=caption,
    box_threshold=box_threshold,
    text_threshold=text_threshold)
    if save:
        os.makedirs('output', exist_ok=True)
        annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=confidence, phrases=phrases)
        cv2.imwrite(os.path.join('output', fname), annotated_frame)
    boxes = torchvision.ops.box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").tolist()
    confidence = confidence.tolist()
    preds = [{
        'Image_ID': fname.split('.')[0],
        'class': 0,  # TODO: replace with actual ReID model results, but for now can just submit bbox and see map, 1: suspect, 0: not suspect
        'confidence': confidence[i],
        'ymin': boxes[i][1],
        'xmin': boxes[i][0],
        'ymax': boxes[i][3],
        'xmax': boxes[i][2]
    } for i in range(len(boxes))]
    preds.sort(key=lambda x: x['confidence'], reverse=True)
    return preds[:4]  # each image only allow 4 boxes at most so take the most confident 4

In [7]:
dataset_root = '../RT-DETR/dataset/test/images'
TEXT_PROMPT = "plushie"
BOX_TRESHOLD = 0.3
TEXT_TRESHOLD = 0.3
results = []

for fname in tqdm.tqdm(os.listdir(dataset_root)):
    results.extend(predict(os.path.join(dataset_root, fname), TEXT_PROMPT, BOX_TRESHOLD, TEXT_TRESHOLD, save=False))

  0%|          | 0/1600 [00:00<?, ?it/s]C:\Program Files\Python39\lib\site-packages\transformers\modeling_utils.py:866: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
C:\Program Files\Python39\lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 1600/1600 [04:46<00:00,  5.58it/s]


In [9]:
import pandas as pd
df = pd.DataFrame(results)
df.to_csv('submission thresh0.3.csv', index=False)

In [11]:
import pandas as pd
df = pd.read_csv('submission thresh0.3.csv')
df['Image_ID'] = df['Image_ID'].apply(lambda x: x.split('.')[0])
df.to_csv('submission thresh0.3.csv', index=False)